# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets, transforms, utils
import os
import random
from collections import defaultdict
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb-api-key")

root_dir = '/kaggle/input/intel-image-classification'

import wandb
wandb.login(key=wandb_api_key)

# Hyperparameters

In [ ]:
batch_size = 32
n_kernels = 16
n_epochs = 100
learning_rate = 1e-3
limit_per_class = 0  # 0 to disable
seed = 42
save_every = 1

# WandB

In [ ]:
model_name = "ConvNet-IntelImage"
wandb.init(
    project="deep-learning",
    config={
        "model": model_name,
        "batch_size": batch_size,
        "n_kernels": n_kernels,
        "n_epochs": n_epochs,
        "learning_rate": learning_rate,
        "seed": seed,
        "limit_per_class": limit_per_class,
    }
)

# IntelImageDataset

In [ ]:
class IntelImageDataset(data.Dataset):
    def __init__(self, root_dir=root_dir, train=True, seed=seed, limit_per_class=limit_per_class):
        super().__init__()

        # params dataset
        self.root_dir = root_dir
        self.train = train
        self.limit_per_class = limit_per_class

        # data and labels
        self.image_paths = []
        self.labels = []

        self.number_of_classes = 0
        self.classes = []

        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(degrees=30), # degrees = range of rotation
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # parameters are ranges
            transforms.RandomGrayscale(p=0.1), # p = probability of applying the transform
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        data_dir = os.path.join(root_dir, 'seg_train/seg_train' if train else 'seg_test/seg_test')

        # Iterate through class folders within the alphabetical folder
        for class_name in os.listdir(data_dir):
            class_dir = os.path.join(data_dir, class_name)
            if not os.path.isdir(class_dir):
                continue  # Skip if not a directory

            filenames = sorted(os.listdir(class_dir))
            count = 0
            for filename in filenames:
                if limit_per_class == 0 or count < limit_per_class:
                    try:
                      img_path = os.path.join(class_dir, filename)

                      Image.open(img_path).verify()

                      self.image_paths.append(img_path)
                      self.labels.append(self.number_of_classes)

                      count += 1

                    except (IOError, SyntaxError):
                      print(
                          'Corrupted image or non-image file detected and skipped:', filename)
                else:
                    break
                    
            self.number_of_classes += 1
            self.classes.append(class_name)

        random.seed(seed)
        combined = list(zip(self.image_paths, self.labels))
        random.shuffle(combined)
        self.image_paths, self.labels = zip(*combined)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]

        try:
          image = Image.open(image_path)
          op = 1
          image = image.convert('RGB')
          rgb = 1
        except (IOError, SyntaxError):
          print(f"Error convert to load {op} RGB {rgb} : {image_path} {label}")

        image = self.transform(image)

        return image, label

# ConvNet

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else :
    device = 'cpu'

print(device)

In [ ]:
class ConvNet(nn.Sequential):
    def __init__(self, input_size, n_kernels, output_size):
        super().__init__(
            nn.Conv2d(in_channels=input_size[0], out_channels=n_kernels, kernel_size=3, padding=1),
            nn.BatchNorm2d(n_kernels),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=n_kernels, out_channels=n_kernels*2, kernel_size=3, padding=1),
            nn.BatchNorm2d(n_kernels*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=n_kernels*2, out_channels=n_kernels*4, kernel_size=3, padding=1),
            nn.BatchNorm2d(n_kernels*4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=n_kernels*4, out_channels=n_kernels*8, kernel_size=3, padding=1),
            nn.BatchNorm2d(n_kernels*8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=n_kernels*8, out_channels=n_kernels*16, kernel_size=3, padding=1),
            nn.BatchNorm2d(n_kernels*16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=n_kernels*16, out_channels=n_kernels*32, kernel_size=3, padding=1),
            nn.BatchNorm2d(n_kernels*32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=n_kernels*32, out_channels=n_kernels*64, kernel_size=3, padding=1),
            nn.BatchNorm2d(n_kernels*64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=n_kernels*64, out_channels=n_kernels*128, kernel_size=3, padding=1),
            nn.BatchNorm2d(n_kernels*128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(),
            nn.Linear(n_kernels*128*int(input_size[1]/(2**8))*int(input_size[2]/(2**8)), 25*output_size),  # Adjust the input size based on your input image size
            nn.ReLU(),
            nn.Linear(25*output_size, output_size)
        )

        for m in self:
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight)
                nn.init.constant_(m.bias, 0)

# Train loop

In [ ]:
def train(model, trainloader, testloader, n_epochs=n_epochs, learning_rate=learning_rate):

    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    train_avg_loss = []
    test_avg_loss = []
    test_accuracy = []

    for i in range(n_epochs):

        print(f"Epoch : {i}")

        train_losses = []
        test_losses = []
        
        # train
        for x, y in trainloader:
            # send to device
            x = x.to(device)
            y = y.to(device)

            # predict
            pred = model(x)
            loss = criterion(pred, y)
            train_losses.append(loss.detach())

            # step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # test
        with torch.no_grad():
            correct = 0

            for x,y in testloader:
                x = x.to(device)
                y = y.to(device)

                pred = model(x)
                loss = criterion(pred, y)
                test_losses.append(loss.detach())

                y_pred = pred.argmax(dim=-1)
                correct = correct + (y_pred==y).sum()

            accuracy = (correct / len(testloader.dataset))

        train_loss = torch.stack(train_losses).mean()
        test_loss = torch.stack(test_losses).mean()

        print(f"train_losses : {train_loss}")
        print(f"test_losses : {test_loss}")
        print(f"accuracy : {accuracy}")
        
        wandb.log({
            "epoch": i,
            "train loss": train_loss,
            "test loss": test_loss,
            "accuracy": accuracy,
        })
        
        if i % save_every == 0:
            torch.save(model.state_dict(), f"epoch_{i}_model.pt")
            wandb.save(f"epoch_{i}_model.pt")

        train_avg_loss.append(train_loss)
        test_avg_loss.append(test_loss)
        test_accuracy.append(accuracy)

    return train_avg_loss, test_avg_loss, test_accuracy

# Create dataset / dataloader

In [ ]:
# Instantiate the train and test set

# train
train_dataset = IntelImageDataset(train=True)

# test
test_dataset = IntelImageDataset(train=False)

In [ ]:
# Instantiate the corresponding data loaders

# train
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# test
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Define Model

In [ ]:
input_features = [3, 256, 256] # Channels (assuming RGB images), Height, Width
output_features = train_dataset.number_of_classes

network = ConvNet(input_size=input_features, n_kernels=n_kernels, output_size=output_features).to(device)
print(network)

# Train Model

In [ ]:
train_avg_loss, test_avg_loss, test_accuracy = train(model=network,
                                                     trainloader=train_loader,
                                                     testloader=test_loader,
                                                     n_epochs=n_epochs,
                                                     learning_rate=learning_rate
                                                     )

# Plot

In [ ]:
train_avg_loss_np = torch.tensor(train_avg_loss).detach().cpu().numpy()
test_avg_loss_np = torch.tensor(test_avg_loss).detach().cpu().numpy()
test_accuracy_np = torch.tensor(test_accuracy).detach().cpu().numpy()

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_avg_loss_np, label='Training Loss')
plt.plot(test_avg_loss_np, label='Testing Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Testing Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(test_accuracy_np, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Test Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Finish wandb run

In [ ]:
# necessary in notebooks
wandb.finish()